In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [2]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
Photosynthesis is the process by which green plants convert sunlight into energy.
Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
The chlorophyll in plant cells captures sunlight during photosynthesis.
Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"}),

    Document(page_content=(
        """Mount Everest is the tallest mountain in the world.
Photosynthesis provides oxygen necessary for human survival.
Climbers from around the world attempt to reach its summit every year despite harsh conditions."""
    ), metadata={"source": "Doc5"}),
]

In [3]:
# create FAISS vector store from the documents
embedding_model = HuggingFaceEmbeddings()
vector_store = FAISS.from_documents(documents=docs, embedding=embedding_model)

c:\Users\dheka\OneDrive\Desktop\11.Langchain_from_beginning\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
base_retriever = vector_store.as_retriever(search_kwargs={"k" : 5})

In [ ]:
# set up the compressor using LLM
llm = ChatHuggingFace(
    llm = HuggingFaceEndpoint(
        repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
        task="text-generation",
        huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")
    )
)

compressor = LLMChainExtractor.from_llm(llm)

In [6]:
# create contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever = base_retriever,
    base_compressor=compressor
)

In [7]:
# Query the retriever
query = "what is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [8]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- {i + 1} ----")
    print(doc.page_content)


--- 1 ----
Photosynthesis is the process by which green plants convert sunlight into energy.

--- 2 ----
Photosynthesis provides oxygen necessary for human survival.

--- 3 ----
Photosynthesis does not occur in animal cells.

--- 4 ----
The chlorophyll in plant cells captures sunlight during photosynthesis.

--- 5 ----
NO_OUTPUT

The context does not provide any information about photosynthesis.
